# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

/Users/Bobo/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/Bobo/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [2]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = len(newsgroups_test.target)
test_data, test_labels = newsgroups_test.data[num_test/2:], newsgroups_test.target[num_test/2:]
dev_data, dev_labels = newsgroups_test.data[:num_test/2], newsgroups_test.target[:num_test/2]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print 'training label shape:', train_labels.shape
print 'test label shape:', test_labels.shape
print 'dev label shape:', dev_labels.shape
print 'labels names:', newsgroups_train.target_names

training label shape: (2034,)
test label shape: (677,)
dev label shape: (676,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


(1) For each of the first 5 training examples, print the text of the message along with the label.

[2 pts]

In [3]:
#def P1(num_examples=5):
### STUDENT START ###

#print train_data[0]
#print train_labels[0]
#print '%s | %s' % (train_data[0], newsgroups_train.target_names[train_labels[0]])

for i in range(5):
    print 'Example %s:\n\nText:\n' % str(i + 1) 
    print train_data[i]
    print '\nLabel: %s\n\n' % newsgroups_train.target_names[train_labels[i]]


### STUDENT END ###
#P1(2)

Example 1:

Text:

Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych

Label: comp.graphics


Example 2:

Text:



Seems to be, barring evidence to the contrary, that Koresh was simply
another deranged fanatic who thought it neccessary to take a whole bunch of
folks with him, children and all, to satisfy his delusional mania. Jim
Jones, circa 1993.


Nope - fruitcakes like Koresh have been demonstrating such evil corruption
for centuries.

(2) Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

[6 pts]

In [241]:
def P2():
### STUDENT START ###

    def get_avg_num_non_zero_features_per_ex():
        return

    def a():
        print '2a.'
        
        # get vocabulary size
        
        vocabulary_size = sparse_matrix.shape[1]
        #vocabulary_size = len(vectorizer.vocabulary_) # alternatively

         
        print '\t The vocabulary contains %s words.' % vocabulary_size 

        # get avg number of non-zero features
        
        num_non_zero_features_per_example = []
        
        # for each example, store the number of non-zero features
        for i in range(sparse_matrix.shape[0]):
            non_zero_features = len(sparse_matrix[i].nonzero()[1])
            
            non_zero_features_for_example_i = \
                non_zero_features / float(vocabulary_size)
            
            num_non_zero_features_per_example.append(
                non_zero_features_for_example_i)

        # calculate the average
        avg_num_non_zero_features = np.mean(num_non_zero_features_per_example)

        print '\t The average number of non-zero features per example is: %5f.' \
            % avg_num_non_zero_features
     
        # get the fraction of non-zero entries
        
        total_entries = sparse_matrix.shape[0] * sparse_matrix.shape[1] 
        fraction_non_zero_entries = sparse_matrix.nnz / float(total_entries)

        print '\t The fraction of non-zero entries is: %5f.' \
            % fraction_non_zero_entries

        #print sparse_matrix[0].nonzero()[1]
        #print sparse_matrix[0,:]
        #sparse_matrix[0,21225]
    
    
    def b():
        print
        print '2b.'
        
        sorted_feature_names_list = sorted(vectorizer.get_feature_names())
        feature_names_list_len = len(sorted_feature_names_list) 
        first_feature_string = sorted_feature_names_list[0]
        last_feature_string = sorted_feature_names_list[feature_names_list_len - 1]
       

        print '\t The first feature string is: ' + first_feature_string
        print '\t The last feature string is: ' + last_feature_string
        
      
    def c():
        print 
        print '2c.'
        
        vocab = ['atheism', 'graphics', 'space', 'religion']
        vocab_size = len(vocab)
        
        vocab_indices = [idx for idx, name in enumerate(feature_names) if 
                    name in vocab]
        
        
        reduced_vocab_matrix = sparse_matrix[:, vocab_indices]
        total_entries = reduced_vocab_matrix.shape[0] * \
            reduced_vocab_matrix.shape[1]
            
        fraction_non_zero_entries = reduced_vocab_matrix.nnz / float(total_entries)
            
        print '\t Considering only four words, the avg number of ' + \
            'non-zero features is: %5f.' % fraction_non_zero_entries    
        #np.array(feature_names)[tuple(np.array(['graphics', 'atheism']))]
        #print feature_names.index(['graphics', 'atheism'])
    
    def d():
        print 
        print '2d.'
        
        vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,2))
        matrix = vectorizer.fit_transform(train_data)
        print len(vectorizer.vocabulary_)
        print vectorizer.get_feature_names()[:50]
        
        vectorizer = CountVectorizer(analyzer='char', ngram_range=(1,3))
        matrix = vectorizer.fit_transform(train_data)
        print len(vectorizer.vocabulary_)
        
    def e():
        print 
        print '2e.'
        
        vectorizer = CountVectorizer(min_df=10)
        matrix = vectorizer.fit_transform(train_data)
        vocab_len = len(vectorizer.vocabulary_)
        
        print '\t With a min document frequency of 10, the vocabulary size is: %s' % vocab_len 
        
    def f():
        print 
        print '2f.'
        
        train_vectorizer = CountVectorizer()
        train_matrix = vectorizer.fit_transform(train_data)
        train_vocab = vectorizer.vocabulary_
        
        dev_vectorizer = CountVectorizer()
        dev_matrix = vectorizer.fit_transform(dev_data)
        dev_vocab = vectorizer.vocabulary_ 
    
        words_only_in_dev = len(set(dev_vocab.keys()) - set(train_vocab.keys()))
        fraction_only_in_dev = words_only_in_dev / float(len(dev_vocab))
        
        print '\t The fraction of words missing from the vocabulary is: %.3f' % fraction_only_in_dev
        
    # instantiate the vectorizer and create the output matrix
    vectorizer = CountVectorizer()
    sparse_matrix = vectorizer.fit_transform(train_data)
    feature_names = vectorizer.get_feature_names()
    
    # answer the posed questions
    a()
    b()
    c()
    d()
    e()
    f()

### STUDENT END ###
P2()

2a.
	 The vocabulary contains 26879 words.
	 The average number of non-zero features per example is: 0.003598.
	 The fraction of non-zero entries is: 0.003598.

2b.
	 The first feature string is: 00
	 The last feature string is: zyxel

2c.
	 Considering only four words, the avg number of non-zero features is: 0.067109.

2d.
221770
[u'00', u'00 10', u'00 12', u'00 13', u'00 21', u'00 22', u'00 30', u'00 47', u'00 50', u'00 52', u'00 advance', u'00 am', u'00 and', u'00 available', u'00 cost', u'00 except', u'00 fax', u'00 have', u'00 hoe', u'00 in', u'00 inc', u'00 international', u'00 it', u'00 jupiter', u'00 lunch', u'00 mov', u'00 non', u'00 noon', u'00 on', u'00 opening', u'00 or', u'00 otherwise', u'00 plus', u'00 pm', u'00 policy', u'00 post', u'00 pounds', u'00 proof', u'00 pulsars', u'00 registration', u'00 saturn', u'00 students', u'00 thank', u'00 the', u'00 to', u'00 type', u'00 ut', u'00 while', u'000', u'000 00']
35557

2e.
	 With a min document frequency of 10, the vocabula

#### (3) Use the default CountVectorizer options and report the f1 score (use metrics.f1_score) for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:  a. Why doesn't nearest neighbors work well for this problem?  b. Any ideas why logistic regression doesn't work as well as Naive Bayes?  c. Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.  [4 pts]

In [61]:
def P3():
### STUDENT START ###

    # initialize the vectorizer
    # and learn the vocabulary
    # from the training data
    vectorizer = CountVectorizer()
    vectorizer.fit(train_data)

    # create matrices for the word counts in 
    # the training and dev data
    train_vector = vectorizer.transform(train_data)
    dev_vector = vectorizer.transform(dev_data)

    def evaluate_default_model(model):
        # instantiate a the classifier and train
        # it on the training data
        model.fit(train_vector, train_labels)
        
        # use the classifier to predict dev labels
        predicted = model.predict(dev_vector)
        
        # calculate the f1 score for each category
        scores = metrics.f1_score(predicted, dev_labels, average=None)
        
        # print the results
        print 'Category \t Score \n'

        for i in range(len(scores)):
            print '{0} \t {1:.3f}'.format(newsgroups_train.target_names[i], scores[i])

        print
        print 'Avg F1 Score: {0:.3f}\n'.format(sum(scores) / len(scores))
    
    def get_best_param(model, params):
        # get the best parameter
        estimator = GridSearchCV(estimator=model, param_grid=params)
        estimator.fit(train_vector, train_labels)

        # predict the labels for the dev data using the best parameter
        predicted = estimator.predict(dev_vector)

        # calculate the F1 score for each category
        f1_scores = metrics.f1_score(predicted, dev_labels, average=None)

        # get the average F1 score across categories
        mean_f1_score = sum(f1_scores) / len(f1_scores)

        # display the results
        param = params.keys()[0]
        print 'The best value for {0} is {1} with a mean F1 score of {2:.3f}'.format(
            param, estimator.best_params_[param], mean_f1_score)
    
    
    print 'F1 Scores for KNN with K = 5: \n'
    evaluate_default_model(model=KNeighborsClassifier())
    
    
    # find the optimal value for K
    # TODO: ASK TODD - best k increases with range (overfitting)?
    # but also...it's worse than above?
    max_k = 25
    params = {'n_neighbors': range(1, max_k + 1)}
    print 'Searching for best K up to {0}...\n'.format(max_k) 
    
    get_best_param(model=KNeighborsClassifier(), params=params)

    # fit a NB
    print
    print 'F1 scores for Multinomial Naive Bayes with alpha = 1:\n'
    
    evaluate_default_model(model=MultinomialNB())
    
    print
    print 'Searching for best alpha...\n'
    alphas = {'alpha': [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]}
    get_best_param(model=MultinomialNB(), params=alphas)  
#     params = {'alpha': alphas}
#     estimator = GridSearchCV(estimator=nb, param_grid=alphas)
#     estimator.fit(train_vector, train_labels)
#     print estimator.best_params_
#     predicted = estimator.predict(dev_vector)
#     #print estimator.score(dev_vector, dev_labels)
#     f1_score = metrics.f1_score(predicted, dev_labels, average=None)
#     print f1_score

#     print 'The best value for alpha is %s \
#     The average score is %s' % (estimator.best_params_['alpha'], sum(f1_score) / len(f1_score))

 
    lr = LogisticRegression()
    lr.fit(train_vector, train_labels)
    lr.score(dev_vector, dev_labels)
    

    print 
    print 'Searching for best regularization strength C...'
    # values from https://stackoverflow.com/a/21818810
    param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
    estimator = get_best_param(model=lr, params=param_grid)
    
### STUDENT END ###
P3()

F1 Scores for KNN with K = 5: 

Category 	 Score 

alt.atheism 	 0.427
comp.graphics 	 0.475
sci.space 	 0.492
talk.religion.misc 	 0.264

Avg F1 Score: 0.415

Searching for best K up to 25...

The best value for n_neighbors is 20 with a mean F1 score of 0.410

F1 scores for Multinomial Naive Bayes with alpha = 1:
Category 	 Score 

alt.atheism 	 0.683
comp.graphics 	 0.918
sci.space 	 0.854
talk.religion.misc 	 0.575

Avg F1 Score: 0.758


Searching for best alpha...

The best value for alpha is 0.01 with a mean F1 score of 0.755

Searching for best regularization strength C...
The best value for C is 0.1 with a mean F1 score of 0.679


ANSWER:

(4) Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

[5 pts]

In [6]:
#def P4():
### STUDENT START ###


### STUDENT END ###
#P4()

ANSWER:

(5) Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

[4 pts]

In [7]:
def empty_preprocessor(s):
    return s

#def better_preprocessor(s):
### STUDENT START ###

### STUDENT END ###

#def P5():
### STUDENT START ###

    
### STUDENT END ###
#P5()

(6) The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.01 (the default is .0001).

[4 pts]

In [8]:
def P6():
    # Keep this random seed here to make comparison easier.
    np.random.seed(0)

    ### STUDENT START ###

    

    ### STUDENT END ###
P6()

(7) Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

[4 pts]

In [11]:
#def P7():
    ### STUDENT START ###



    ### STUDENT END ###
#P7()

ANSWER:

(8) EXTRA CREDIT

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.

- [1 pt] for a reasonable attempt
- [2 pts] for improved performance